In [1]:
import cv2
import numpy as np
from scipy import ndimage
from matplotlib import pyplot as plt
photo = cv2.imread("test.jpeg")
# converting image to grayscale
gray_scale = cv2.cvtColor(photo, cv2.COLOR_BGR2GRAY)


In [3]:

def LDP_1st(I):
    # LDP_1st returns 1st-order derivation on 4 directions: 0, 45, 90 and 135 degree
    # I: the grayscaled image

    h = I.shape[0]  # height of the image
    w = I.shape[1]  # width of the image

    # filter for horizontal direction (0 degree)
    h1 = np.array([[0, 0, 0], [0, 1, -1], [0, 0, 0]])
    # filter for diagonal direction (45 degree)
    h2 = np.array([[0, 0, -1], [0, 1, 0], [0, 0, 0]])
    # filter for vertical direction (90 degree)
    h3 = np.array([[0, -1, 0], [0, 1, 0], [0, 0, 0]])
    # filter for diagonal direction (135 degree)
    h4 = np.array([[-1, 0, 0], [0, 1, 0], [0, 0, 0]])

    F = np.hstack([ndimage.convolve(I, h1), ndimage.convolve(I, h2), ndimage.convolve(
        I, h3), ndimage.convolve(I, h4)])  # concatenate the filtered images horizontally

    return F


In [4]:
def LDP_2nd(I):
    # LDP_2nd returns the second-order derivation on 4 directions: 0, 45, 90 and 135 degrees
    # I: the grayscaled image

    h = I.shape[0]  # height of the image
    w = I.shape[1]  # width of the image

    # filter for horizontal direction
    h1 = np.array([[0, 0, 0], [0, 1, -1], [0, 0, 0]])
    # filter for diagonal direction (450 degree)
    h2 = np.array([[0, 0, -1], [0, 1, 0], [0, 0, 0]])
    # filter for vertical direction
    h3 = np.array([[0, - 1, 0], [0, 1, 0], [0, 0, 0]])
    # filter for diagonal direction (135 degree)
    h4 = np.array([[-1, 0, 0], [0, 1, 0], [0, 0, 0]])

    F_11st = LDP_1st(I)

    # concatenate the filtered images horizontally
    F = np.hstack([ndimage.convolve(F_11st[:, 1:w], h1), ndimage.convolve(F_11st[:, w+1:2*w], h2),
                  ndimage.convolve(F_11st[:, 2*w+1:3*w], h3), ndimage.convolve(F_11st[:, 3*w+1:4*w], h4)])

    return F

In [5]:
def LDP_2nd_hist(I):
    # LDP_2nd_hist returns the unscaled histogram containing 2^8 bins of the input image
    # I: the grayscaled image

    h = I.shape[0]  # height of the image
    w = I.shape[1]  # width of the image

    hist = np.zeros(1024)  # 256 bins for each angle

    # F = LDP_2nd(I)
    F = LDP_1st(I)

    F = F > 0

    # binary pattern for each pixel
    bin_pattern = np.array([[1, 2, 4], [128, 0, 8], [64, 32, 16]])

    for k in range(4):  # alpha = 0, 45, 90, 135
        for i in range(1, h-1):  # ignore the borders
            for j in range(1, w-1):  # ignore the borders
                blk_j = k*w + j

                pattern = (F[i-1:i+2, (blk_j-1):(blk_j+2)] - F[i][blk_j]) != 0

                pattern = np.sum(pattern * bin_pattern)  # 0 <= pattern <=255

                index = k*256 + pattern
                hist[index] += 1

    return hist

In [8]:
hist = LDP_2nd_hist(gray_scale)
print(hist)

TypeError: 'numpy.ndarray' object is not callable